# Advent Of Code 2022

In [1]:
using System.IO;

The below script needs to be able to find the current output cell; this is an easy method to get it.

## Day 1

In [ ]:
IEnumerable<int> ReadCalories() {
    var total = 0;
    foreach(var line in File.ReadLines("../data/in-1.txt")) {
        if(int.TryParse(line, out var value)) {
            total += value;
        }
        else {
            yield return total;
            total = 0;
        }
    }
    yield return total;
}

Console.WriteLine(ReadCalories().Max());

# Day 3

In [ ]:
public class Day3 {

    int GetPriority(string str) 
    {
        long contains = 0;
        var half = str.Length / 2;
        for(var i=0; i<half; i++) 
        {
            var prio = GetPriority(str[i]);
            var mask = 1L << prio;
            contains |= mask;
        }
        for(var i=0; i<half; i++) 
        {
            var prio = GetPriority(str[i + half]);
            var mask = 1L << prio;
            if((contains & mask) > 1) {
                return prio;
            }
        }
        return 0;
    }
    
    int GetPriority(char c) 
    {
        if(c > 'Z') 
        {
            return (int)(c - 'a') + 1;
        }
        return (int)(c - 'A') + 27;
    }
    
    int Run3_1(string file) {
        return
            File
            .ReadLines(file)
            .Select(l => GetPriority(l))
            .Sum();
    }

    int GetPriority(string str1, string str2, string str3) 
    {
        var chars = str1.Distinct().Concat(str2.Distinct()).Concat(str3.Distinct());
        var c = 
            chars
            .GroupBy(c => c)
            .Single(g => g.Count() == 3)
            .Key;
        return GetPriority(c);
    }
    
    
    IEnumerable<int> Run3_2(string file) {
        var lines = File.ReadLines(file).ToList();
        for(var i=0; i<lines.Count; i+=3) {
            yield return GetPriority(lines[i], lines[i+1], lines[i+2]);
        }
    }
    

    public void Run() {
        Console.WriteLine(Run3_1("../data/in-3.txt"));
        Console.WriteLine(Run3_2("../data/in-3.txt").Sum());
    }
}

new Day3().Run();


# Day 4

In [ ]:
public class Day4 {
    public class Range {
        public Range(int from, int to) {
            if(to < from) {
                throw new ArgumentException();
            }
            this.From = from;
            this.To = to;
        }

        public int From;
        public int To;

        public bool Contains(int n) => From <= n && n <= To;

        public bool FullyContains(Range other) => Contains(other.From) && Contains(other.To);

        public bool Overlaps(Range other) {
            if(other.To < this.From) {
                return false;
            }
            if(other.From > this.To) {
                return false;
            }
            return true;
        }

        public static Range Parse(string str) {
            var parts = str.Split('-');
            return new Range(int.Parse(parts[0]), int.Parse(parts[1]));
        }
    }

    public int Run1(string file) {
        var count = 0;
        foreach(var line in File.ReadAllLines(file)) {
            var parts = line.Split(',');
            var range1 = Range.Parse(parts[0]);
            var range2 = Range.Parse(parts[1]);
            if(range1.FullyContains(range2) || range2.FullyContains(range1)) {
                count++;
            }
        }
        return count;
    }

    public int Run2(string file) {
        var count = 0;
        foreach(var line in File.ReadAllLines(file)) {
            var parts = line.Split(',');
            var range1 = Range.Parse(parts[0]);
            var range2 = Range.Parse(parts[1]);
            if(range1.Overlaps(range2)) {
                count++;
            }
        }
        return count;
    }


    public void Run() {
        Console.WriteLine(Run1("../data/in-4.txt"));
        Console.WriteLine(Run2("../data/in-4.txt"));
    }
}

new Day4().Run();

# Day 5


In [ ]:
public class Day5 {

    public class Yard {
        public Yard(int size) {
            for(var i=0; i<size; i++)  {
                Stacks.Add(new());
            }
        }
        public List<LinkedList<char>> Stacks = new ();

        public void Move(Move move, int mode) {
            if(mode == 9000) {
                Move9000(move);
            }
            if(mode == 9001) {
                Move9001(move);
            }
        }

        public void Move9000(Move move) {
            for(var i=0; i<move.Quantity; i++) {
                var item = Stacks[move.From].First.Value;
                Stacks[move.From].RemoveFirst();
                Stacks[move.To].AddFirst(item);
            }
        }

        public void Move9001(Move move) {
            var items = new List<char>();            
            for(var i=0; i<move.Quantity; i++) {
                var item = Stacks[move.From].First.Value;
                Stacks[move.From].RemoveFirst();
                items.Add(item);
            }
            for(var i=0; i<move.Quantity; i++) {
                Stacks[move.To].AddFirst(items[items.Count - i - 1]);
            }
        }

        public void Print() {
            for(var i=0; i<Stacks.Count; i++) {
                Console.Write($"{i + 1}");
                foreach(var c in Stacks[i].Reverse()) {
                    Console.Write($" [{c}]");
                }
                Console.WriteLine();
            }
        }

        public string State() {
            var result = new StringBuilder();
            foreach(var stack in Stacks) {
                if(stack.Any()) {
                    result.Append(stack.First());
                }
            }
            return result.ToString();
        }
    }

    public class Move {
        public int Quantity;
        public int From;
        public int To;
        public static Move Parse(string str) {
            var parts = str.Split(' ');
            return new Move() {
                Quantity = int.Parse(parts[1]),
                From = int.Parse(parts[3]) - 1,
                To = int.Parse(parts[5]) - 1,
            };
        }
    }

    public void Run(string file, int mode) {
        var lines = File.ReadAllLines(file);
        var i = 0;
        string line;
        do {
            line = lines[i++];
        } while(!string.IsNullOrEmpty(line));
        var size = lines[0].Length;
        var yard = new Yard(size / 3);
        for(var l = 0; l < i - 1; l++) {
            for(var x = 0; x < size; x++) {
                var index = x * 4 + 1;
                if(index < lines[l].Length) {
                    var c = lines[l][index];
                    if(c >= 'A' && c <= 'Z') {
                        yard.Stacks[x].AddLast(c);
                    }
                }
            }
        }
        yard.Print();
        Console.WriteLine();
        
        while(i < lines.Length) {
            line = lines[i++];
            var move = Move.Parse(line);
            yard.Move(move, mode);
        }
        yard.Print();
        Console.WriteLine();
        Console.WriteLine($"End state: {yard.State()}");
        Console.WriteLine();
    }


}

//new Day5().Run("../data/in-5.txt", 9000);
new Day5().Run("../data/in-5.txt", 9001);


# Day 6

In [ ]:
public class Day6
{

    public const int MarkerSize = 14;
    private int Parse(string line) {
        var buffer = new char[MarkerSize];
        for(var i=0; i<line.Length; i++) {
            buffer[i % MarkerSize] = line[i];
            if(i >= MarkerSize) {
                if(!ContainsDuplicate(buffer)) {
                    Console.WriteLine($"Found marker: {string.Join("", buffer)}");
                    return i + 1;
                }
            }
        }
        return line.Length + 1;
    }

    private bool ContainsDuplicate(char[] buffer) {
        var set = new HashSet<char>(buffer);
        return set.Count != MarkerSize;
    }


    public void Run(string file) {
        foreach(var line in File.ReadAllLines(file)) {
            Console.WriteLine(Parse(line));
        }
    }

}

new Day6().Run("../data/in-6.txt");

# Day 7

In [ ]:
public class Day7 {

    public class Directory {
        public string Name;
        public Directory Parent;
        public Dictionary<string, Directory> Directories = new();
        public Dictionary<string, long> Files = new();
        public bool Visited;
        public long TotalSize() => Directories.Sum(d => d.Value.TotalSize()) + Files.Sum(f => f.Value);


        public IEnumerable<Directory> All() {
            yield return this;
            foreach(var kv in Directories) {
                foreach(var a in kv.Value.All()) {
                    yield return a;
                }
            }
        }


        public void Print(StringBuilder builder, string name, int indent) {
            if(Visited) {
                throw new InvalidOperationException("already visited");
            }
            Visited = true;
            builder.Append("".PadLeft(indent * 2));
            builder.AppendLine($"{name} (dir)");
            foreach(var dir in Directories) {
                dir.Value.Print(builder, dir.Key, indent + 1);
            }
            foreach(var file in Files) {
                builder.Append("".PadLeft(indent * 2));
                builder.AppendLine($"  {file.Key} (file, size = {file.Value})");
            }
        }    
    }

    public void Run(string file) {
        var lines = File.ReadAllLines(file).GetEnumerator();
        Directory currentDir = null;
        Directory root = null;
        while(lines.MoveNext()) {
            if(lines.Current.ToString().StartsWith("$ ")) {
                var parts = lines.Current.ToString().Substring(2).Split(' ');
                if(parts[0] == "cd") {
                    if(parts[1] == "/") {
                        currentDir = root = new Directory() { Name = "/" };
                    }
                    else if(parts[1] == "..") {
                        if(currentDir.Parent == null) {
                            throw new InvalidOperationException();
                        }
                        else {
                            currentDir = currentDir.Parent;
                        }
                    }
                    else {
                        currentDir = currentDir.Directories[parts[1]];
                    }
                }
                else if(parts[0] == "ls") {
                    
                }
            }
            else {
                var parts2 = lines.Current.ToString().Split(' ');
                if(parts2[0] == "dir") {
                    currentDir.Directories.Add(parts2[1], new Directory() {
                        Name = parts2[1],
                        Parent = currentDir
                    });
                }
                else {
                    currentDir.Files.Add(parts2[1], long.Parse(parts2[0]));
                }
            }
        }

        var builder = new StringBuilder();
        root.Print(builder, "/", 0);
        //Console.WriteLine(builder.ToString());

        var total = 
            root
            .All()
            .Where(d => d.TotalSize() <= 100000)
            .Sum(d => d.TotalSize());
        Console.WriteLine(total);
        
        var unused = 70000000 - root.TotalSize();
        var needed = 30000000 - unused;
        Console.WriteLine($"We need to free up {needed} bytes");
        var best = 
            root
            .All()
            .Where(d => d.TotalSize() >= needed)
            .OrderBy(d => d.TotalSize())
            .First();
        Console.WriteLine($"We should pick {best.Name}, increasing total size by {best.TotalSize()}");

    }
}

new Day7().Run("../data/in-7.txt");

# Day 8

In [4]:
public class Day8 {

    public class Forest {
        public Forest(string[] data) {
            _data = data;
            _width = data[0].Length;
            _height = data.Length;
        }
        private readonly int _width, _height;
        private readonly string[] _data;

        public IEnumerable<int> GetHeights(int x, int y, int dx, int dy) {
            while(true) {
                x += dx;
                y += dy;
                if(x < 0 || x >= _width) {
                    yield break;
                }
                if(y < 0 || y >= _height) {
                    yield break;
                }
                yield return GetHeight(x, y);
            }
        }

        public int GetHeight(int x, int y) => (_data[y][x] - '0');

        public bool IsVisible(int x, int y) {
            if(x == 0 || x == _width - 1) {
                return true;
            }
            if(y == 0 || y == _height - 1) {
                return true;
            }

            var height = GetHeight(x, y);
            foreach(var (dx, dy) in GetVector2is()) {
                var visible = true;
                foreach(var h in GetHeights(x, y, dx, dy)) {
                    if(h >= height) {
                        visible = false;
                        break;
                    }
                }
                if(visible) {
                    //Console.WriteLine($"Found inner tree: {x}, {y}");
                    return true;
                }
            }
            return false;
        }


        public int GetScenicScore(int x, int y) {
            var score = 1;
            var builder = new StringBuilder();
            builder.Append($"ViewDistance ({x}, {y}):");
            foreach(var (dx, dy) in GetVector2is()) {
                var distance = GetViewingDistance(x, y, dx, dy);
                builder.Append($"* {distance}");
                score *= distance;
                
            }
            builder.Append($" = {score}");
            //Console.WriteLine(builder.ToString());
            return score;
        }

        public int GetViewingDistance(int x, int y, int dx, int dy) {
            var distance = 0;
            var height = GetHeight(x, y);
            while(true) {
                x += dx;
                y += dy;
                if(x < 0 || x >= _width) {
                    return distance;
                }
                if(y < 0 || y >= _height) {
                    return distance;
                }
                distance++;
                if(GetHeight(x, y) >= height) {
                    return distance;
                }   
            }
        }

        public IEnumerable<(int, int)> GetVector2is() {
            yield return (0, -1);
            yield return (-1, 0);
            yield return (0, 1);
            yield return (1, 0);
        }

        public int CountTreesVisibleFromOutside() {
            var result = 0;
            for(var x=0; x<_width; x++) {
                for(var y=0; y<_height; y++) {
                    if(IsVisible(x, y)) {
                        result++;
                    }
                }
            }
            return result;
        }

        public int MaximumScenicScore() {
            var score = 0;
            for(var x=1; x<_width - 1; x++) {
                for(var y=1; y<_height - 1; y++) {
                    score = Math.Max(score, GetScenicScore(x, y));
                }
            }
            return score;
        }
    }



    public void Run1(string file) {
        var lines = File.ReadAllLines(file);
        var forest = new Forest(lines);
        Console.WriteLine(forest.CountTreesVisibleFromOutside());
    }

    public void Run2(string file) {
        var lines = File.ReadAllLines(file);
        var forest = new Forest(lines);
        Console.WriteLine(forest.MaximumScenicScore());        
    }

}

new Day8().Run1("../data/in-8.txt");
new Day8().Run2("../data/in-8.txt");

1851
574080


# Day 9

In [11]:
public class Day9 {

    public record struct Vector2i(int X, int Y) {
        public static Vector2i Parse(char c) {
            return c switch {
                'U' => new Vector2i(0, -1),
                'D' => new Vector2i(0, 1),
                'L' => new Vector2i(-1, 0),
                'R' => new Vector2i(1, 0),
                _ => throw new ArgumentOutOfRangeException()
            };
        }

        public double Distance(Vector2i other) => Math.Sqrt(X * other.X + Y * other.Y);

        public Vector2i Add(Vector2i other) => new Vector2i(X + other.X, Y + other.Y);

        public static Vector2i Min(Vector2i v1, Vector2i v2) => new Vector2i(Math.Min(v1.X, v2.X), Math.Min(v1.Y, v2.Y));

        public static Vector2i Max(Vector2i v1, Vector2i v2) => new Vector2i(Math.Max(v1.X, v2.X), Math.Max(v1.Y, v2.Y));

        public static IEnumerable Vector2i All(Vector2i v1, Vector2i v2) {
            var min = Min(v1, v2);
            var max = Max(v1, v2);
            for(var y=min.Y; y<=max.Y; y++) {
                for(var x=min.X; x<=max.X; x++) {
                    yield return new(x, y);
                }    
            }
        }

        public static IEnumerable<Vector2i> All() => All(new(-1, -1), new(1, 1));

        public static readonly Vector2i Zero = new();

    }

    public class Game {
        public Game() {
            _visited.Add(new Vector2i());
        }

        private readonly HashSet<Vector2i> _visited = new ();
        private Vector2i _head;
        private Vector2i _tail;

        public void Move(string[] moves) {
            foreach(var move in moves) {
                Move(move);
            }
        }

        public void Move(string move) {
            // Decode move
            Console.WriteLine($"Applying move: {move}");
            var Vector2i = Vector2i.Parse(move[0]);
            var amount = int.Parse(move.Substring(2));
            for(var i=0; i<amount; i++) {
                Move(Vector2i);
            }
            Print();
            Console.WriteLine();
        }

        public void Move(Vector2i direction) {
            _head = _head.Add(direction);
            var distance = _head.Distance(_tail);
            if(distance >= 1) {
                var tailDirection = 
                    Vector2i
                    .All()
                    .OrderBy(d => _tail.Add(d).Distance(_head))
                    .ThenBy(d => d.Distance(direction))
                    .First();
                _tail = _tail.Add(tailDirection);
                _visited.Add(_tail);
            }            
        }

        public int PlacesVisited() => _visited.Count;


        public void Print() {
            var builder = new StringBuilder();
            var min = Vector2i.Max()


            var min = Vector2i.Min(new(), Vector2i.Min(_head, _tail));
            var max = Vector2i.Max(new(), Vector2i.Min(_head, _tail));
            var prevY = min.Y;
            foreach(var pos in Vector2i.All(min, max)) {
                if(pos == _head) {
                    builder.Append('H');
                }
                else if(pos == _tail) {
                    builder.Append('T');
                }
                else if(pos == Vector2i.Zero) {
                    builder.Append('s');
                }
                else {
                    builder.Append('.');
                }
                
                if(prevY != min.Y) {
                    prevY = min.Y;
                    builder.Append(Environment.NewLine);
                }
                    
            }

            Console.WriteLine(builder.ToString());
        }
    }

    public void Run1(string file) {
        var lines = File.ReadAllLines(file);
        var game = new Game();
        game.Move(lines);
        Console.WriteLine(game.PlacesVisited());
    }

}

new Day9().Run1("../data/test-9.txt");


(22,44): error CS1002: ; expected

(85,37): error CS1002: ; expected



Error: compilation error

: 

: 